In [ ]:
# install a bunch of stuff
!pip3 install pandas
!pip3 install scikit-learn
!pip3 install matplotlib
!pip3 install plotly
!pip3 install rpy2

In [ ]:
# import a bunch of stuff
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
import numpy as np
from scipy import stats
from sklearn.preprocessing import MinMaxScaler as mms

In [ ]:
# install other things
!pip3 install pyqt5

In [ ]:
# install other things
!pip3 install bioinfokit

In [ ]:
# install other things
!pip3 install virtualenv

In [ ]:
# read data from file into table
table = pd.read_csv("data/salmon.merged.gene_counts.tsv", sep = "\t")

In [ ]:
# print headers of columns
print(table.head(0))

In [ ]:
# extract names column
genes = table.iloc[:, 0]

#calculate averages of each row
gene_avg = table.iloc[:, 2:].mean(axis=1)

#create new table
avg_table = pd.DataFrame({'Genes':genes, 'Averages':gene_avg})
avg_table

In [ ]:
# table of ranges
# find range
gene_min = table.iloc[:, 2:].min(axis=1)
gene_max = table.iloc[:, 2:].max(axis=1)
gene_range = gene_max - gene_min

# create new table
range_table = pd.DataFrame({'Genes':genes, 'Range':gene_range})
range_table

In [ ]:
# table of averages of genes
# identify columns based sample type
oh_cols = table.filter(regex=r'^old_human').columns
oo_cols = table.filter(regex=r'^old_organoid').columns
yh_cols = table.filter(regex=r'^young_human').columns
yo_cols = table.filter(regex=r'^young_organoid').columns

# get averages of each gene of each sample
oh_avg = table[oh_cols].mean(axis=1)
oo_avg = table[oo_cols].mean(axis=1)
yh_avg = table[yh_cols].mean(axis=1)
yo_avg = table[yo_cols].mean(axis=1)

# put into data frame
better_avg_table = pd.DataFrame({
    'Gene':genes,
    'Old Human':oh_avg,
    'Old Organoid':oo_avg,
    'Young Human':yh_avg,
    'Young Organoid':yo_avg
})

better_avg_table

In [ ]:
# write ranges into new data frame
# old human range
oh_min = table[oh_cols].min(axis=1)
oh_max = table[oh_cols].max(axis=1)
oh_range = oh_max - oh_min

# old organoid range
oo_min = table[oo_cols].min(axis=1)
oo_max = table[oo_cols].max(axis=1)
oo_range = oo_max - oo_min

# young human range
yh_min = table[yh_cols].min(axis=1)
yh_max = table[yh_cols].max(axis=1)
yh_range = yh_max - yh_min

# young organoid range
yo_min = table[yo_cols].min(axis=1)
yo_max = table[yo_cols].max(axis=1)
yo_range = yo_max - yo_min

# put into data frame
better_range_table = pd.DataFrame({'Gene':genes, 'Old Human':oh_range, 'Old Organoid':oo_range, 'Young Human':yh_range, 'Young Organoid':yo_range})
better_range_table.to_csv("data.csv")

In [ ]:
# remove gene name columns so they don't interfere with PCA
#table = table.drop(columns=['gene_name', 'gene_id'])

In [ ]:
# pca of transformed table
pca = PCA()

# find line of least regression
pca.fit(table.T)

# make x-y coords
transformed_data = pca.transform(table.T)
transformed_table = pd.DataFrame(data=transformed_data, columns=['PC{}'.format(i+1) for i in range(transformed_data.shape[1])])

transformed_table = transformed_table.set_index(table.columns)

transformed_table

In [ ]:
# not sure, s/t with pca
pca.explained_variance_ratio_

In [ ]:
# make a scatter table
scatter_table = go.Scatter(
    x = transformed_table[transformed_table.columns[6]],
    y = transformed_table[transformed_table.columns[7]],
    text = transformed_table.index,
    mode = 'markers'
)

# 
layout = go.Layout(
    xaxis=dict(title=transformed_table.columns[6]),
    yaxis=dict(title=transformed_table.columns[7]),
    hovermode='closest'
)

figure = go.Figure(data=[scatter_table], layout=layout)

pyo.plot(figure, filename='scatter_plot.html')

In [ ]:
# heatmap of 229 relevant genes
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
from scipy.stats import f_oneway
import numpy as np
from bioinfokit import analys, visuz
import math


# extract names column
table = pd.read_csv("data/salmon.merged.gene_counts.tsv", sep = "\t")
genes = table.iloc[:, 0]

# identify columns based sample type
oo_cols = table.filter(regex=r'^old_organoid').columns
tm_cols = table.filter(regex=r'^treatment_met').columns
tn_cols = table.filter(regex='^treatment_nmn').columns
tr_cols = table.filter(regex='^treatment_res').columns
yo_cols = table.filter(regex=r'^young_organoid').columns


# get averages of each gene of each sample
oo_avg = table[oo_cols].mean(axis=1)
tm_avg = table[tm_cols].mean(axis=1)
tn_avg = table[tn_cols].mean(axis=1)
tr_avg = table[tr_cols].mean(axis=1)
yo_avg = table[yo_cols].mean(axis=1)

#sort heat data
heat_data = {
    'Old Organoid':oo_avg,
    'Metformin Treatment':tm_avg,
    'NMN Treatment':tn_avg,
    'Resveratrol Treatment':tr_avg,
    'Young Organoid':yo_avg
}

# put into data frame
heat_table = pd.DataFrame(heat_data)
heat_table.index = genes
num_genes_left = 1000
heat_table = heat_table.head(num_genes_left)
heat_table

heat_table =(np.log10(heat_table))

# Calculate the standard deviation of each row
row_std = heat_table.apply(np.std, axis=1)

# Set a threshold for the minimum standard deviation
threshold = 0.03

# Filter out rows with standard deviation below the threshold
filtered_heat_table = heat_table[row_std >= threshold]

# Get the indices of the filtered rows
filtered_indices = filtered_heat_table.index

# Remove the filtered rows from the original heat_table
heat_table = heat_table.drop(filtered_indices)

# heatmap with hierarchical clustering 
#visuz.gene_exp.hmap(df=heat_table, dim=(3, 6), tickfont=(6, 4))

#print(heat_table.shape[0])

# heatmap without hierarchical clustering 
visuz.gene_exp.hmap(df=heat_table, rowclus=False, colclus=False, dim=(10, 30), tickfont=(9, 8), show=True)


In [ ]:
# sort data by sample type
oo_data = heat_table.columns['Old Organoid']
tm_data = heat_table.columns['Metformin Treatment']
tn_data = heat_table.columns['NMN Treatment']
tr_data = heat_table.columns['Resveratrol Treatment']
yo_data = heat_table.columns['Young Organoid']


In [ ]:
# line graph of 229 genes expressed
# Define the width and height of the figure
fig_width = 120
fig_height = 12

# Create a new figure with the specified size
plt.figure(figsize=(fig_width, fig_height))

# Get the column names from heat_table
column_names = heat_table.columns

# Plot line graphs for each column
for column in column_names:
    plt.plot(heat_table.index, heat_table[column], label=column)

# Set plot title and labels
plt.title('Column Comparison')
plt.xlabel('Row')
plt.ylabel('Value')

# Rotate the x-axis tick labels
plt.xticks(rotation=45)

# Add legend
plt.legend()

# Show the plot
plt.show()


In [ ]:
# seaborn plot comparing samples to each other
import seaborn as sns

# Generate the scatter plot matrix
sns.pairplot(heat_table)

# Show the plot
plt.show()

In [ ]:
# correlation heatmap
# Calculate the correlation matrix
correlation_matrix = heat_table.corr()

# Plot the correlation matrix as a heatmap
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm")

# Show the plot
plt.show()

In [ ]:
# failed attempt at normalized heatmap
import pandas as pd
from pandas.plotting import parallel_coordinates

# Create a copy of the heat_table DataFrame
normalized_heat_table = heat_table.copy()

# Normalize the values in the DataFrame to a common scale
normalized_heat_table = (normalized_heat_table - normalized_heat_table.min()) / (normalized_heat_table.max() - normalized_heat_table.min())

# Add a target column for coloring the lines (optional)
normalized_heat_table['target'] = range(len(normalized_heat_table))

# Create the parallel coordinates plot
parallel_coordinates(normalized_heat_table, 'target', colormap='coolwarm')

# Show the plot
plt.show()

In [ ]:
# another seaborn plot
import seaborn as sns
import matplotlib.pyplot as plt

# Create a grid of scatter plots
sns.pairplot(heat_table, diag_kind='kde')

# Show the plot
plt.show()

In [ ]:
# line plot of 229 genes in heatmap, w/ x markers
plt.plot(heat_table, marker='x')

In [ ]:
# graph to figure out threshold of relevant genes
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
from scipy.stats import f_oneway
import numpy as np
from bioinfokit import analys, visuz
import math


# extract names column
table = pd.read_csv("data/salmon.merged.gene_counts.tsv", sep = "\t")
genes = table.iloc[:, 0]

# identify columns based sample type
oo_cols = table.filter(regex=r'^old_organoid').columns
tm_cols = table.filter(regex=r'^treatment_met').columns
tn_cols = table.filter(regex='^treatment_nmn').columns
tr_cols = table.filter(regex='^treatment_res').columns
yo_cols = table.filter(regex=r'^young_organoid').columns


# get averages of each gene of each sample
oo_avg = table[oo_cols].mean(axis=1)
tm_avg = table[tm_cols].mean(axis=1)
tn_avg = table[tn_cols].mean(axis=1)
tr_avg = table[tr_cols].mean(axis=1)
yo_avg = table[yo_cols].mean(axis=1)

#sort heat data
heat_data = {
    'Old Organoid':oo_avg,
    'Metformin Treatment':tm_avg,
    'NMN Treatment':tn_avg,
    'Resveratrol Treatment':tr_avg,
    'Young Organoid':yo_avg
}

# put into data frame
heat_table = pd.DataFrame(heat_data)
heat_table.index = genes

#heat_table = heat_table.head(num_genes_left)
#heat_table

heat_table =(np.log10(heat_table))

# Calculate the standard deviation of each row
row_std = heat_table.apply(np.std, axis=1)

# Store number of genes left
xlens = []
ylens = []

# Set a threshold for the minimum standard deviation
for threshold in np.arange(0.0, 0.1, 0.000001):

    # Filter out rows with standard deviation below the threshold
    filtered_heat_table = heat_table[row_std <= threshold]
    xlength = filtered_heat_table.shape[0]
    xlens.append(xlength)
    ylens.append(threshold)

print(len(xlens), len(ylens))

plt.scatter(xlens, ylens)


In [ ]:
# failed graph?
table = pd.read_csv("data/salmon.merged.gene_counts.tsv", sep = "\t")

# List of important genes to include in heatmap
hm_genes = ['ATP6', 'ATP8', 'CBX5', 'CCND1', 'CDK4', 'CDK6', 'CDKN1A', 'CDKN2A', 'CDKN2A', 'CHD3', 'CHD4', 'CHD5', 'COX1', 'COX2', 'COX3', 'CXCL5', 'CYTB', 'GCP-1', 'GCP-2', 'GM-CSE', 'GROa', 'ICAM1', 'IKBKA', 'IKBKG', 'IL6', 'IL7R', 'MCP-1', 'MDM2', 'ND1', 'ND2', 'ND3', 'ND4', 'ND4L', 'ND5', 'ND6', 'NFKB1', 'NFKB2', 'OPG', 'POT1', 'PTGES2', 'RB1', 'RELA', 'TERF1', 'TERF2', 'TERT', 'TIMP2', 'TP53', 'bFGF', 'uPAR']
#hm_genes = sorted(hm_genes) #sort genes

# identify columns based sample type
oo_cols = table.filter(regex=r'^old_organoid').columns
tm_cols = table.filter(regex=r'^treatment_met').columns
tn_cols = table.filter(regex='^treatment_nmn').columns
tr_cols = table.filter(regex='^treatment_res').columns
yo_cols = table.filter(regex=r'^young_organoid').columns


# get averages of each gene of each sample
oo_avg = table[oo_cols].mean(axis=1)
tm_avg = table[tm_cols].mean(axis=1)
tn_avg = table[tn_cols].mean(axis=1)
tr_avg = table[tr_cols].mean(axis=1)
yo_avg = table[yo_cols].mean(axis=1)

#sort heat data
heatmap_data = {
    'Old Organoid':oo_avg,
    'Metformin Treatment':tm_avg,
    'NMN Treatment':tn_avg,
    'Resveratrol Treatment':tr_avg,
    'Young Organoid':yo_avg
}

# put into data frame
filtered_df = table[table['gene_id'].isin(hm_genes)]

heatmap = pd.DataFrame(filtered_df)
heatmap.index = hm_genes
num_genes = len(hm_genes)
#heatmap = heatmap.head(num_genes)
#heatmap

heatmap =(np.log10(heatmap))

"""# Calculate the standard deviation of each row
row_std = heatmap.apply(np.std, axis=1)

# Set a threshold for the minimum standard deviation
threshold = 0.03

# Filter out rows with standard deviation below the threshold
filtered_heatmap = heatmap[row_std >= threshold]

# Get the indices of the filtered rows
filtered_hm_indices = filtered_heatmap.index

# Remove the filtered rows from the original heat_table
heatmap = heatmap.drop(filtered_hm_indices)
"""

# heatmap with hierarchical clustering 
#visuz.gene_exp.hmap(df=heat_table, dim=(3, 6), tickfont=(6, 4))

#print(heatmap.shape[0])

# heatmap without hierarchical clustering 
visuz.gene_exp.hmap(df=heatmap, rowclus=False, colclus=False, dim=(10, 20), tickfont=(9, 8), show=True)
"""
heat_table = pd.DataFrame(heat_data)
heat_table.index = genes

#heat_table = heat_table.head(num_genes_left)
#heat_table

heat_table = (np.log10(heat_table))

# Calculate the standard deviation of each row
row_std = heat_table.apply(np.std, axis=1)
"""

In [ ]:
# heatmap data
heatmap

In [ ]:
# heatmaps specific to gene pathways, pre-normalization
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from bioinfokit import analys, visuz

# Read the data
table = pd.read_csv("data/salmon.merged.gene_counts.tsv", sep = "\t")
genes = table.iloc[:, 0]

# remove gene name columns so they don't interfere with PCA
#table = table.drop(columns=['gene_name', 'gene_id'])

# List of important genes to include in the heatmap
#hm_genes = ['ATP6', 'ATP8', 'CBX5', 'CCND1', 'CDK4', 'CDK6', 'CDKN1A', 'CDKN2A', 'CDKN2A', 'CHD3', 'CHD4', 'CHD5', 'COX1', 'COX2', 'COX3', 'CXCL5', 'CYTB', 'GCP-1', 'GCP-2', 'GM-CSE', 'GROa', 'ICAM1', 'IKBKA', 'IKBKG', 'IL6', 'IL7R', 'MCP-1', 'MDM2', 'ND1', 'ND2', 'ND3', 'ND4', 'ND4L', 'ND5', 'ND6', 'NFKB1', 'NFKB2', 'OPG', 'POT1', 'PTGES2', 'RB1', 'RELA', 'TERF1', 'TERF2', 'TERT', 'TIMP2', 'TP53', 'bFGF', 'uPAR']
hm_genes = ['TP53', 'CDK6', 'CDK4', 'CDKN1A', 'CDKN2A', 'RB1', 'MDM2', 'CDKN2A', 'CCND1', 'RELA', 'NFKB2', 'NFKB1', 'IKBKG', 'IKBKA', 'IL6', 'IL7R', 'CXCL5', 'PTGES2', 'GCP-2', 'ICAM1', 'TIMP2', 'GM-CSE', 'bFGF', 'GROa', 'GCP-1', 'uPAR', 'MCP-1', 'OPG', 'ATP6', 'ATP8', 'ND2', 'ND1', 'ND4', 'ND4L', 'COX1', 'ND5', 'ND6', 'CYTB', 'ND3', 'COX3', 'COX2', 'TERT', 'TERF1', 'TERF2', 'POT1', 'CHD5', 'CHD3', 'CHD4', 'CBX5']
hm_group1 = ['TP53', 'CDK6', 'CDK4', 'CDKN1A', 'CDKN2A', 'RB1', 'MDM2', 'CDKN2A', 'CCND1']
hm_group2 = ['RELA', 'NFKB2', 'NFKB1', 'IKBKG', 'IKBKA']
hm_group3 = ['IL6', 'IL7R', 'CXCL5', 'PTGES2', 'GCP-2', 'ICAM1', 'TIMP2', 'GM-CSE', 'bFGF', 'GROa', 'GCP-1', 'uPAR', 'MCP-1', 'OPG']
hm_group4 = ['ATP6', 'ATP8', 'ND2', 'ND1', 'ND4', 'ND4L', 'COX1', 'ND5', 'ND6', 'CYTB', 'ND3', 'COX3', 'COX2']
hm_group5 = ['TERT', 'TERF1', 'TERF2', 'POT1']
hm_group6 = ['CHD5', 'CHD3', 'CHD4', 'CBX5']

# Identify columns based on sample type
oo_cols = table.filter(regex=r'^old_organoid').columns
tm_cols = table.filter(regex=r'^treatment_met').columns
tn_cols = table.filter(regex='^treatment_nmn').columns
tr_cols = table.filter(regex='^treatment_res').columns
yo_cols = table.filter(regex=r'^young_organoid').columns

# Get averages of each gene for each sample type
oo_avg = table[oo_cols].mean(axis=1)
tm_avg = table[tm_cols].mean(axis=1)
tn_avg = table[tn_cols].mean(axis=1)
tr_avg = table[tr_cols].mean(axis=1)
yo_avg = table[yo_cols].mean(axis=1)

# Sort heat data
heat_data = {
    'Old Organoid': oo_avg,
    'Metformin Treatment': tm_avg,
    'NMN Treatment': tn_avg,
    'Resveratrol Treatment': tr_avg,
    'Young Organoid': yo_avg
}
# print(heat_data)

# Create the heat table with the important genes
heatmap = pd.DataFrame(heat_data)
#heatmap.set_index(pd.Index(genes), inplace=True)
heatmap.index = genes
heatmap = heatmap.reindex(hm_genes)
heatmap = heatmap.dropna()

# remove gene name columns so they don't interfere with PCA
#table = table.drop(columns=['gene_name', 'gene_id'])

# Log-transform the data
heatmap = np.log10(heatmap)

heatmap = heatmap[heatmap.index.isin(hm_genes)]
heatmap_group1 = heatmap[heatmap.index.isin(hm_group1)]
heatmap_group2 = heatmap[heatmap.index.isin(hm_group2)]
heatmap_group3 = heatmap[heatmap.index.isin(hm_group3)]
heatmap_group4 = heatmap[heatmap.index.isin(hm_group4)]
heatmap_group5 = heatmap[heatmap.index.isin(hm_group5)]
heatmap_group6 = heatmap[heatmap.index.isin(hm_group6)]
# 
#heatmap = heatmap.dropna()

# Generate the heatmap without hierarchical clustering
visuz.gene_exp.hmap(df=heatmap_group1, figname='Heatmap Group 1', rowclus=False, colclus=False, dim=(5, 6), tickfont=(9, 8), show=True)
visuz.gene_exp.hmap(df=heatmap_group2, figname='Heatmap Group 2', rowclus=False, colclus=False, dim=(5, 6), tickfont=(9, 8), show=True)
visuz.gene_exp.hmap(df=heatmap_group3, figname='Heatmap Group 3', rowclus=False, colclus=False, dim=(5, 6), tickfont=(9, 8), show=True)
#visuz.gene_exp.hmap(df=heatmap_group4, figname='Heatmap Group 4', rowclus=False, colclus=False, dim=(5, 6), tickfont=(9, 8), show=True)
visuz.gene_exp.hmap(df=heatmap_group5, figname='Heatmap Group 5', rowclus=False, colclus=False, dim=(5, 6), tickfont=(9, 8), show=True)
visuz.gene_exp.hmap(df=heatmap_group6, figname='Heatmap Group 6', rowclus=False, colclus=False, dim=(5, 6), tickfont=(9, 8), show=True)



In [ ]:
# scaling heatmap data
#heatmap_scaled = (heatmap.max() - heatmap)/(heatmap.max() - heatmap.min())
#heatmap_scaled

# Create a MinMaxScaler object
scaler = mms(feature_range=(0, 1))

# Scale the DataFrame row by row
scaled_data = scaler.fit_transform(heatmap.values.T).T

# Create a new DataFrame with scaled values
scaled_df = pd.DataFrame(scaled_data, columns=heatmap.columns, index=heatmap.index)

print("Scaled DataFrame:")
print(scaled_df.head(26))

In [ ]:
# heatmaps specific to gene pathways, post-normalization
#heatmap_scaled = heatmap_scaled[heatmap_scaled.index.isin(hm_genes)]
heatmap_scaled = scaled_df[scaled_df.index.isin(hm_genes)]
heatmap_scaled_group1 = heatmap_scaled[heatmap_scaled.index.isin(hm_group1)]
heatmap_scaled_group2 = heatmap_scaled[heatmap_scaled.index.isin(hm_group2)]
heatmap_scaled_group3 = heatmap_scaled[heatmap_scaled.index.isin(hm_group3)]
heatmap_scaled_group4 = heatmap_scaled[heatmap_scaled.index.isin(hm_group4)]
heatmap_scaled_group5 = heatmap_scaled[heatmap_scaled.index.isin(hm_group5)]
heatmap_scaled_group6 = heatmap_scaled[heatmap_scaled.index.isin(hm_group6)]
# 
#heatmap = heatmap.dropna()

# Generate the heatmap without hierarchical clustering
visuz.gene_exp.hmap(df=heatmap_scaled_group1, figname='Heatmap Group 1', rowclus=False, colclus=False, dim=(5, 6), tickfont=(9, 8), show=True)
visuz.gene_exp.hmap(df=heatmap_scaled_group2, figname='Heatmap Group 2', rowclus=False, colclus=False, dim=(5, 6), tickfont=(9, 8), show=True)
visuz.gene_exp.hmap(df=heatmap_scaled_group3, figname='Heatmap Group 3', rowclus=False, colclus=False, dim=(5, 6), tickfont=(9, 8), show=True)
#visuz.gene_exp.hmap(df=heatmap_group4, figname='Heatmap Group 4', rowclus=False, colclus=False, dim=(5, 6), tickfont=(9, 8), show=True)
visuz.gene_exp.hmap(df=heatmap_scaled_group5, figname='Heatmap Group 5', rowclus=False, colclus=False, dim=(5, 6), tickfont=(9, 8), show=True)
visuz.gene_exp.hmap(df=heatmap_scaled_group6, figname='Heatmap Group 6', rowclus=False, colclus=False, dim=(5, 6), tickfont=(9, 8), show=True)
